In [ ]:
using KomaMRI, MAT, Plots, LaTeXStrings, Measures

seq = read_seq("sequences/mpf_001_PhantomStudy_short.seq")

function simulate_slice_profile_x(
    seq::Sequence; x=range(-2.e-2, 2.e-2, 200), sim_params=Dict{String,Any}("Δt_rf" => 1e-6)
)
    sim_params["return_type"] = "state"
    sys = Scanner()
    obj = Phantom{Float64}(; x=Array(x), z=zeros(size(x)))
    mag = simulate(obj, seq, sys; sim_params)
    return mag
end

In [ ]:
z = -0.01:0.000001:0.01
M2 = simulate_slice_profile_x(seq[2]; x=z)
M1035 = simulate_slice_profile_x(seq[1035]; x=z)
q = Plots.plot(z, M2.z)
Plots.plot!(q, z, abs.(M1035.xy))

In [ ]:
half = 2.5
ideal_trans = [ (x >= -half && x <= half) ? 1.0 : 0.0 for x in z*1000 ]

p1 = plot(z*1000, abs.(M1035.xy),
    label = "Excitation",
    color = :black,
    lw = 2,
    xlabel = "Position (mm)",
    ylabel = L"|\mathbf{M}_{\mathit{xy}}|",
    legend = :right,
    guidefont = font(8),
    legendfont = font(6),
    grid = false
)

plot!(p1, z*1000, ideal_trans,
    label = "Nominal",
    color = :red,
    linestyle = :dash,
    dashes = (3,6),
    lw = 1
)

ideal_long = [ (x >= -half && x <= half) ? -1.0 : 1.0 for x in z*1000 ]

p2 = plot(z*1000, M2.z,
    label = "Inversion",
    color = :black,
    lw = 2,
    xlabel = "Position (mm)",
    ylabel = L"\mathbf{M}_{\mathit{z}}",
    legend = :right,
    guidefont = font(8),
    legendfont = font(6),
    grid = false
)

plot!(p2, z*1000, ideal_long,
    label = "Nominal",
    color = :red,
    linestyle = :dash,
    dashes = (3,6),
    lw = 1
)

plot(p2, p1,
    layout = (1, 2),
    size = (800, 300),
    left_margin = 4mm,
    bottom_margin = 4mm
)

fig = plot(p2, p1,
    layout = (1, 2),
    size = (800, 300),
    left_margin = 6mm,
    bottom_margin = 6mm
)
Plots.savefig(fig, "slice_profile.pdf")
